# Implementarea unui Sistem Expert pentru Animale

Un exemplu din [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

În acest exemplu, vom implementa un sistem simplu bazat pe cunoștințe pentru a determina un animal pe baza unor caracteristici fizice. Sistemul poate fi reprezentat prin următorul arbore AND-OR (acesta este doar o parte din arborele complet, putem adăuga cu ușurință mai multe reguli):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.ro.png)


## Propriul nostru shell pentru sisteme expert cu inferență inversă

Să încercăm să definim un limbaj simplu pentru reprezentarea cunoștințelor bazat pe reguli de producție. Vom folosi clase Python ca cuvinte cheie pentru a defini regulile. Vor exista, în esență, 3 tipuri de clase:
* `Ask` reprezintă o întrebare care trebuie adresată utilizatorului. Conține setul de răspunsuri posibile.
* `If` reprezintă o regulă și este doar o sintaxă simplificată pentru a stoca conținutul regulii.
* `AND`/`OR` sunt clase care reprezintă ramuri AND/OR ale arborelui. Ele doar stochează lista de argumente în interior. Pentru a simplifica codul, toată funcționalitatea este definită în clasa părinte `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

În sistemul nostru, memoria de lucru ar conține lista de **fapte** ca **perechi atribut-valoare**. Baza de cunoștințe poate fi definită ca un dicționar mare care asociază acțiuni (fapte noi care ar trebui inserate în memoria de lucru) cu condiții, exprimate ca expresii AND-OR. De asemenea, unele fapte pot fi `Întrebate`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Pentru a realiza inferența inversă, vom defini clasa `Knowledgebase`. Aceasta va conține:
* `Memoria` de lucru - un dicționar care mapă atributele la valori
* `Regulile` bazei de cunoștințe în formatul definit mai sus

Două metode principale sunt:
* `get` pentru a obține valoarea unui atribut, efectuând inferența dacă este necesar. De exemplu, `get('color')` ar obține valoarea unui slot de culoare (va întreba dacă este necesar și va stoca valoarea pentru utilizare ulterioară în memoria de lucru). Dacă cerem `get('color:blue')`, va întreba pentru o culoare și apoi va returna valoarea `y`/`n` în funcție de culoare.
* `eval` efectuează inferența propriu-zisă, adică parcurge arborele AND/OR, evaluează sub-obiectivele etc.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Acum să definim baza noastră de cunoștințe despre animale și să efectuăm consultarea. Rețineți că acest apel vă va pune întrebări. Puteți răspunde tastând `y`/`n` pentru întrebările de tip da-nu sau specificând un număr (0..N) pentru întrebările cu răspunsuri multiple mai lungi.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Utilizarea PyKnow pentru inferență înainte

În următorul exemplu, vom încerca să implementăm inferența înainte folosind una dintre bibliotecile pentru reprezentarea cunoștințelor, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** este o bibliotecă pentru crearea sistemelor de inferență înainte în Python, concepută să fie similară cu vechiul sistem clasic [CLIPS](http://www.clipsrules.net/index.html).

Am fi putut implementa și noi lanțul înainte fără prea multe probleme, dar implementările naive nu sunt, de obicei, foarte eficiente. Pentru o potrivire mai eficientă a regulilor, se folosește un algoritm special, [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Vom defini sistemul nostru ca o clasă care extinde `KnowledgeEngine`. Fiecare regulă este definită printr-o funcție separată cu adnotarea `@Rule`, care specifică momentul în care regula ar trebui să fie declanșată. În interiorul regulii, putem adăuga noi fapte folosind funcția `declare`, iar adăugarea acestor fapte va duce la apelarea unor reguli suplimentare de către motorul de inferență înainte.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Odată ce am definit o bază de cunoștințe, populăm memoria noastră de lucru cu câteva fapte inițiale și apoi apelăm metoda `run()` pentru a efectua inferența. Puteți observa ca rezultat că noi fapte deduse sunt adăugate în memoria de lucru, inclusiv faptul final despre animal (dacă am configurat corect toate faptele inițiale).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
